In [1]:
import tensorflow as tf
from tensorflow.layers import dense,batch_normalization
from tensorflow.nn import conv2d,max_pool,pool,avg_pool,leaky_relu
import numpy as np

In [2]:
def ConvBlock(input_tensor,filters,kernal_size,stride=1,isActivation=True):
    with tf.name_scope('conv'):
        padding = 'SAME'
        input_shape = input_tensor.get_shape().as_list() [-1]
        w = tf.Variable(tf.random_normal([kernal_size,kernal_size,input_shape,filters]))
        x = conv2d(input_tensor, w, strides=[1,stride,stride,1], padding=padding)
        x = batch_normalization(x)
        if isActivation:
            x = leaky_relu(x)
        return x

def PoolingLayer(input_tensor):
    m = max_pool(input_tensor,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')
    return m

def Residual(input_tensor,filters, kernal_size, block_count, first_stride=2):
    if(len(filters) != len(kernal_size)) :
        raise ValueError("filters count and kernal_size count must be the same(Filter:%d, Kernal: %d)"%(len(filters), len(kernal_size)))
    if (len(kernal_size)==0 or len(filters)==0):
        raise ValueError("Have not entered value")
    shortcut = input_tensor
    x = input_tensor
    
    residual_conv = True
    
    for i in range(block_count):
        with tf.name_scope("Block"):
            
            x = ConvBlock(x,filters[0],kernal_size=kernal_size[0])
            
            if(len(filters)==2):
                x = ConvBlock(x,filters[1],kernal_size=kernal_size[1],stride=first_stride,isActivation=False)
            elif( len(filters)==3):
                x = ConvBlock(x,filters[1],kernal_size=kernal_size[1])
                x = ConvBlock(x,filters[2],kernal_size=kernal_size[2],stride=first_stride, isActivation=False)

            if residual_conv:                
                if first_stride!=1:
                    shortcut = ConvBlock(shortcut,filters[-1],kernal_size=kernal_size[-1], stride=first_stride, isActivation=False)
                    first_stride=1
                else:
                    shortcut = ConvBlock(shortcut,filters[-1],kernal_size=kernal_size[-1], stride=first_stride, isActivation=False)                    
                residual_conv = False
            x = x+shortcut
            x = leaky_relu(x)
        shortcut = x
    return x

def ResNet50(input_size=(None,224,224,3)):    
    with tf.name_scope('Input'):
        input_shape = input_size
        x = tf.placeholder('float32',input_shape)
        y =  tf.placeholder('float32',input_shape)    
    with tf.name_scope('conv1'):
        x = ConvBlock(x,64,7,2)
    with tf.name_scope('conv2'):
        x = PoolingLayer(x)
        x = Residual(x,[64,64,256],[1,3,1],3,1)
    with tf.name_scope('conv3'):
        x = Residual(x,[128,128,512],[1,3,1],4)
    with tf.name_scope('conv4'):
        x = Residual(x,[256,256,1024],[1,3,1],6)
    with tf.name_scope('conv5'):
        x = Residual(x,[512,512,2048],[1,3,1],3)
        return x
    
def ResNet101(input_size=(None,224,224,3)):    
    with tf.name_scope('Input'):
        input_shape = input_size
        x = tf.placeholder('float32',input_shape)
        y =  tf.placeholder('float32',input_shape)    
    with tf.name_scope('conv1'):
        x = ConvBlock(x,64,7,2)
    with tf.name_scope('conv2'):
        x = PoolingLayer(x)
        x = Residual(x,[64,64,256],[1,3,1],3,1)
    with tf.name_scope('conv3'):
        x = Residual(x,[128,128,512],[1,3,1],4)
    with tf.name_scope('conv4'):
        x = Residual(x,[256,256,1024],[1,3,1],23)
    with tf.name_scope('conv5'):
        x = Residual(x,[512,512,2048],[1,3,1],3)
        return x
    
def main():
    graph = ResNet50()

if __name__ == "__main__":
    main()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
